In [ ]:
import os
import xml.etree.ElementTree as ET

file_append = []

for subdir, dirs, files in os.walk(r'../Train-corpus'):
    for filename in files:
        filepath = subdir + os.sep + filename
        mytree = ET.parse(filepath)
        root = mytree.getroot()

        # Handled mw tags
        for element in root.iter('mw'):
            mw_words = ""

        # strip the text field
            for words in element.iter('w'):
                mw_words += words.text
            tag = element.attrib.get('c5')
            file_append.append(mw_words.strip() + "_" + tag)

        # Handled w tags
        for element in root.iter('w'):
            word = element.text
            tag = element.attrib.get('c5')
            tag_list = tag.split('-')
            for tags in tag_list:
                file_append.append(word.strip() + "_" + tags)

        # Handled c tags
        for element in root.iter('c'):
            file_append.append(element.text.strip() + "_" + element.attrib.get('c5'))


# mw tags i.e. multiword there are tags init
# c tags i.e. puctuations
# if tag1-tag2 then word_tag1, word_tag2
# Handle the spacing 
# Don't take words from hw tags

In [ ]:
myDict = {}

for word in file_append:
    if (word in myDict):
        myDict[word] += 1
    else:
        myDict[word] = 1


In [ ]:
import operator

word_dict = {}
tag_dict = {}

for element in file_append:
    temp = element.split("_")
    word = temp[0].strip()
    tag = temp[1]

    if (word in word_dict):
        word_dict[word] += 1
    else: 
        word_dict[word] = 1

    if (tag in tag_dict):
        tag_dict[tag] += 1
    else: 
        tag_dict[tag] = 1

word_dict_ordered = dict( sorted(word_dict.items(), key=operator.itemgetter(1),reverse=True))
tag_dict_ordered = dict( sorted(tag_dict.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
probability_of_word_given_tag = {}

# P(Word|Tag) = P(Word_Tag)/P(Tag) = Count(Word_Tag)/Count(Tag)

for word in word_dict.keys():
    for tag in tag_dict.keys():
        temp = word+"_"+tag
        temp1 = word+"|"+tag
        if (temp in myDict):
            probability_of_word_given_tag[temp1] = myDict[temp] / tag_dict[tag]
        else:
            probability_of_word_given_tag[temp1] = 0

In [ ]:
def predict( word ):
    
    if word not in word_dict:
        return "UNC"
    mx_p = -1
    tag_gen = ""
    for tags in tag_dict.keys():
        word_given_tag = word + "|" + tags
        P = 0
        if word in word_dict:
            P = (float(probability_of_word_given_tag[word_given_tag]) * int(
                tag_dict[tags])) / int(word_dict[word])
        if P > mx_p:
            mx_p = P
            tag_gen = tags
    return tag_gen


In [ ]:
import glob
test_files = glob.glob("../Test-corpus/*/*.xml")
print(test_files[0])

In [ ]:
ref = []
tagged = []

correct = 0
total = 0
mw_correct = 0
mw_total = 0
p_correct = 0
p_total = 0

for test_file in test_files:
    mytree1 = ET.parse(test_file)
    root1 = mytree1.getroot()

    """Normal words tag"""
    for element in root1.iter('w'):
        word = element.text.strip()
        tag = element.attrib.get('c5')

        tag_gen = predict(word)
        if tag == tag_gen:
            correct += 1

        if tag.find('-') != -1:
            if tag.split('-')[0] == tag_gen or tag.split('-')[1] == tag_gen:
                correct += 1
                ref.append(tag_gen)
            else:
                ref.append(tag)
        else:
            ref.append(tag)
        total += 1
        tagged.append(tag_gen)


    """Multi-words"""
    for element in root1.iter("mw"):
        word = ""
        for word_mw in element.iter('w'):
            word += word_mw.text
        if word[-1] == " ":
            word = word[0:-1]
        tag = element.attrib.get('c5')
        tag_gen = predict(word)
        if tag == tag_gen:
            mw_correct += 1

        if tag.find('-') != -1:
            if tag.split('-')[0] == tag_gen or tag.split('-')[1] == tag_gen:
                mw_correct += 1
                ref.append(tag_gen)
            else:
                ref.append(tag)
        else:
            ref.append(tag)
        mw_total += 1
        tagged.append(tag_gen)

    """Punctuations"""
    for element in root1.iter("c"):
        try:
            word = element.text.strip()
            tag = element.attrib.get('c5')
            tag_gen = predict(word)
            if tag == tag_gen:
                p_correct += 1

            if tag.find('-') != -1:
                if tag.split('-')[0] == tag_gen or tag.split('-')[1] == tag_gen:
                    p_correct += 1
                    ref.append(tag_gen)
                else:
                    ref.append(tag)
            else:
                ref.append(tag)
            p_total += 1
            tagged.append(tag_gen)
        except:
            print("")

correct += mw_correct + p_correct
total += mw_total + p_total
print(correct,total)
print(correct/total)

# Correct words: 3818421
# Total words: 4159247
# 0.9180558403961102

In [ ]:
from nltk.metrics import ConfusionMatrix
import sys 

stdoutOrigin=sys.stdout 
sys.stdout = open("../generatedFiles/Confusion_matrix.txt", "w")
cm = ConfusionMatrix(ref, tagged)
print(cm)
sys.stdout.close()
sys.stdout=stdoutOrigin

In [ ]:
from collections import Counter  
labels = list(tag_dict.keys())
labels.append('UNC')

true_positives = Counter()
false_negatives = Counter()
false_positives = Counter()

for i in labels:
    for j in labels:
        if i == j:
            true_positives[i] += cm[i,j]
        else:
            false_negatives[i] += cm[i,j]
            false_positives[j] += cm[i,j]

print ("TP:", sum(true_positives.values()))
print ("FN:", sum(false_negatives.values()))
print ("FP:", sum(false_positives.values()))
TP=sum(true_positives.values())
FP=sum(false_positives.values())
FN=sum(false_negatives.values())


precision = TP / float(TP+FP)
recall = TP / float(TP+FN)
fscore = 2 * (precision * recall) / float(precision + recall)
print( fscore , precision , recall)

In [ ]:
import csv
tag_csv=csv.reader(open("../generatedFiles/tag_freq.csv",'r',encoding="utf8"))
Tag_list=[]
for tags in tag_csv:
    Tag_list.append(tags[0])
tag_number={}
tag_number['<S>']=0
tag_number['<E>']=0
i=1
for tags in Tag_list:
    tag_number[tags]=i
    i+=1

In [ ]:
import numpy as np 
transition_matrix=np.zeros((62,62))


In [ ]:
def increment(curr,prev):
    if '-' in prev:
        compound_prev=prev.split('-')
        if '-' in curr:
            compound_curr=curr.split('-')
            transition_matrix[tag_number[compound_prev[0]]][tag_number[compound_curr[0]]]+=1
            transition_matrix[tag_number[compound_prev[0]]][tag_number[compound_curr[1]]]+=1
            transition_matrix[tag_number[compound_prev[1]]][tag_number[compound_curr[0]]]+=1
            transition_matrix[tag_number[compound_prev[1]]][tag_number[compound_curr[1]]]+=1
        else:
            transition_matrix[tag_number[compound_prev[0]]][tag_number[curr]]+=1
            transition_matrix[tag_number[compound_prev[1]]][tag_number[curr]]+=1
    else:
        if '-' in curr:
            compound_curr=curr.split('-')
            transition_matrix[tag_number[prev]][tag_number[compound_curr[0]]]+=1
            transition_matrix[tag_number[prev]][tag_number[compound_curr[1]]]+=1
        else:
            transition_matrix[tag_number[prev]][tag_number[curr]]+=1
            

In [ ]:
import glob
import xml.etree.ElementTree as ET 
files = glob.glob("../Train-corpus/*/*.xml")
for filepath in files:
    root=ET.parse(filepath).getroot()
    for s_tag in root.iter('s'):
        prev='<S>'
        curr=''
        end='<E>'
        for tags in s_tag:
            if 'c5' not in tags.attrib:
                for u_tag in tags:
                    if 'c5' in tags.attrib:
                        curr=u_tag.attrib.get('c5')
                        increment(curr,prev)
                        prev=curr
            elif tags.tag == 'mw':
                for w in tags.iter('w'):
                    curr = w.attrib.get('c5')
                    increment(curr, prev)
                    prev = curr
            else:
                curr=tags.attrib.get('c5')
                increment(curr,prev)
                prev=curr
        increment(end,prev)

In [ ]:
print(transition_matrix[41])

In [ ]:
for row in range(len(transition_matrix)):
    s=np.sum(transition_matrix[row])
    if s!=0.0:
        for i in range(len(transition_matrix[row])):
            transition_matrix[row][i]/=s

In [ ]:
print(Tag_list)
print(tag_number)

In [ ]:
def Viterbi(word_list):
    pred_tag_list = []
    prev_prob_list = []
    is_start = True
    for word in word_list:
        curr_prob_list = np.zeros(61)
        if is_start:
            for i in range(61):
                word_given_tag = word+"|"+Tag_list[i]
                curr_prob_list[i] = transition_matrix[0][i+1]
                try:
                    curr_prob_list[i] *= probability_of_word_given_tag[word_given_tag]
                except:
                    curr_prob_list[i] = 0
                if max(curr_prob_list) == 0:
                    prev_prob_list = np.ones(61)
            is_start = False   
        else:
            for i in range(61):
                word_given_tag = word+"|"+Tag_list[i]
                for j in range(61):
                    curr_prob_list[i] = max(curr_prob_list[i], prev_prob_list[j]*transition_matrix[j+1][i+1])
                try:
                    curr_prob_list[i] *= probability_of_word_given_tag[word_given_tag]
                except:
                    curr_prob_list[i] = 0
        if max(curr_prob_list) != 0:                    
            pred_tag = Tag_list[np.argmax(curr_prob_list)]
            # Below change is temp
            prev_prob_list = curr_prob_list
        else:
            pred_tag = 'UNC'
        pred_tag_list.append(pred_tag)
        # prev_prob_list = curr_prob_list
    return pred_tag_list


In [ ]:
import glob
import xml.etree.ElementTree as ET 
files = glob.glob("../Train-corpus/*/*.xml")
# for filepath in files:
ref = []
tagged = []
total = 0
correct = 0

for filepath in files:
    root=ET.parse(filepath).getroot()
    for s_tag in root.iter('s'):
        word_list = []
        tag_list = []
        for tags in s_tag:
            if 'c5' not in tags.attrib:
                try:
                    for u_tag in tags:
                        curr_tag = u_tag.attrib.get('c5')
                        curr_word = u_tag.text.strip()
                        tag_list.append(curr_tag)
                        word_list.append(curr_word)
                except:
                    continue
            elif tags.tag == 'mw':
                for w in tags.iter('w'):
                    try:
                        curr_tag = w.attrib.get('c5')
                        curr_word = w.text.strip()
                        tag_list.append(curr_tag)
                        word_list.append(curr_word)
                    except:
                        continue
            else:
                try:
                    curr_tag = tags.attrib.get('c5')
                    curr_word = tags.text.strip()
                    tag_list.append(curr_tag)
                    word_list.append(curr_word)
                except:
                    continue

        pred_tag_list = Viterbi(word_list)

        for i in range(len(tag_list)):
            curr_tag = tag_list[i]
            try:
                if "-" in curr_tag:
                    mc_list = curr_tag.split("-")
                    if mc_list[0] ==pred_tag_list[i] or mc_list[1] ==pred_tag_list[i]:
                        correct += 1
                        ref.append(pred_tag_list[i])
                    else:
                        ref.append(curr_tag)
                elif pred_tag_list[i] == tag_list[i]:
                    correct += 1
                    ref.append(pred_tag_list[i])
                else:
                    ref.append(curr_tag)
                tagged.append(pred_tag_list[i])
                total += 1
            except:
                print(filepath)
                continue
print(correct, total, correct/total)

In [ ]:
print(pred_tag_list)
print(word_list)
print(tag_list)

In [ ]:
print(Viterbi(['My', 'name', 'is', 'saksham', 'asdfdsfsd', 'sdssdfsdsdfdsf', '.']))

In [ ]:
from nltk.metrics import ConfusionMatrix
import sys 

stdoutOrigin=sys.stdout 
sys.stdout = open("../generatedFiles/Confusion_matrix_HMM.txt", "w")
cm = ConfusionMatrix(ref, tagged)
print(cm)
sys.stdout.close()
sys.stdout=stdoutOrigin